<a href="https://colab.research.google.com/github/WhiteAndBlackFox/nlp/blob/RNN/%D0%A0%D0%B5%D0%BA%D1%83%D1%80%D1%80%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B5_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BD%D1%8B%D0%B5_%D1%81%D0%B5%D1%82%D0%B8_RNN_LSTM_GRU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Рекуррентные нейронные сети RNN LSTM GRU

## Доставляем и импортируем библиотеки

In [2]:
!pip install -U tensorflow pandas xlrd
!pip install stop_words pymorphy2 tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.4 kB/s 
     |████████████████████████████████| 96 kB 5.8 MB/s 
     |████████████████████████████████| 5.8 MB 43.0 MB/s 
     |████████████████████████████████| 438 kB 63.2 MB/s 
     |████████████████████████████████| 1.6 MB 46.8 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninsta

In [42]:
import pandas as pd
import numpy as np
import re

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking,MaxPooling1D, BatchNormalization
from keras.preprocessing.text import Tokenizer
from keras.callbacks import TensorBoard, EarlyStopping
from keras.losses import SparseCategoricalCrossentropy

import nltk
from nltk.probability import FreqDist
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
for val in ['stopwords', 'punkt']:
  nltk.download(val)

import warnings

warnings.filterwarnings('ignore')

from tqdm import tqdm

tqdm.pandas()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Дополнительные функции

In [52]:
def clean_text(text):
  """
    Функция для предобработки текста
  """
  # Удаляем все имена пользователей
  text = re.sub("@[\w]*", "", str(text).lower())
  # Удаляем всю пунктуацию
  text = "".join(c for c in text if c not in punct)
  # Заменим спецсимволы, числа и английские слова на пробелы.
  text = re.sub("[^а-яёА-ЯЁ\_]", " ", text)
  # Получаем токены
  text = word_tokenize(text)
  # Удаляем стоп слова
  text = [w for w in text if w not in stopwordslist]
  # Нормализация текста
  text = [morpher.parse(word)[0].normal_form for word in text if word not in stopwordslist]
  return " ".join(text)

def create_model(name_arch, rnn_method=None):
  """
    Функция для создания модели по имени архитектуры
  """
  model = Sequential([
                      Embedding(input_dim=word_count,
                                input_length=training_length,
                                output_dim=30,
                                trainable=True,
                                mask_zero=True),
                      Masking(mask_value=0.0),
  ])
  if name_arch == 'CNN':
    model.add(Conv1D(128, 3, activation="relu"))
    model.add(GlobalMaxPool1D())
    model.add(Dense(10, activation="relu"))
  
  elif name_arch == 'CNN+RNN':
    #
    if rnn_method is None:
      raise ValueError("rnn_method doesn't empty in architecture CNN+RNN")
    
    model.add(Conv1D(32, 3, activation="relu"))
    model.add(Conv1D(16, 2, activation="relu"))
    model.add(MaxPooling1D(1))
    model.add(rnn_method(16, return_sequences=True))
    model.add(rnn_method(16))
    model.add(Dense(32, activation="relu"))

  elif name_arch == 'RNN+CNN':
    #
    if rnn_method is None:
      raise ValueError("rnn_method doesn't empty in architecture RNN+CNN")
    
    model.add(rnn_method(32, return_sequences=True))
    model.add(rnn_method(32, return_sequences=True))
    model.add(Conv1D(32, 3, activation="relu"))
    model.add(MaxPooling1D(2))
    model.add(Activation("relu"))
    model.add(Conv1D(16, 3, activation="relu"))
    model.add(GlobalMaxPool1D())
    model.add(Dense(64, activation="relu"))

  else:
    model.add(name_arch(64))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.5))
  
  model.add(Dense(1, activation='sigmoid'))
  model.compile(optimizer='adam', 
                loss='binary_crossentropy', 
                metrics=['accuracy'])
  return model

def fit_evalute_model(model, x_train, y_train, x_test, y_test, callbacks=None):
  """
    Функция для обучения и предсказания
  """
  model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=verbose,
          validation_split=validation_split)
  
  score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=verbose, callbacks=callbacks)
  print('Test score:', score[0])
  print('Test accuracy:', score[1])
  
  return score

## Подготовим данные

In [4]:
#@title Глобальные переменные

stopwordslist = stopwords.words("russian")
morpher = MorphAnalyzer()
punct = set(punctuation)

In [7]:
!wget -O 'data.zip' -qq --no-check-certificate https://gbcdn.mrgcdn.ru/uploads/asset/2800985/attachment/a68dea3b4a742be353077d807a6f4a8d.zip
!unzip 'data.zip'
!rm 'data.zip'

Archive:  data.zip
replace data/val.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/val.csv            
  inflating: data/test.csv           
  inflating: data/train.csv          


In [8]:
df_train = pd.read_csv("data/train.csv")
df_val = pd.read_csv("data/val.csv")

In [13]:
df_train['preprocess_text'] = df_train['text'].progress_apply(clean_text)
df_val['preprocess_text'] = df_val['text'].progress_apply(clean_text)

100%|██████████| 22683/22683 [00:41<00:00, 552.73it/s]


In [14]:
df_train.head()

,id,text,class,preprocess_text
0,0,@alisachachka не уезжаааааааай. :(❤ я тоже не ...,0,уезжаааааааать хотеть уезжать
1,1,RT @GalyginVadim: Ребята и девчата!\nВсе в кин...,1,ребята девчата кино это любовь сегодня завтра ...
2,2,RT @ARTEM_KLYUSHIN: Кто ненавидит пробки ретви...,0,ненавидеть пробка ретвит
3,3,RT @epupybobv: Хочется котлету по-киевски. Зап...,1,хотеться котлета покиевск запретный плод
4,4,@KarineKurganova @Yess__Boss босапопа есбоса н...,1,босапоп есбосой бояться мороз


In [35]:
tokenizer = Tokenizer(num_words=None, 
                      filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                      lower = False,
                      split = ' ')
tokenizer.fit_on_texts(df_train['preprocess_text'].values)

sequences_train = tokenizer.texts_to_sequences(df_train['preprocess_text'].values)
sequences_val = tokenizer.texts_to_sequences(df_val['preprocess_text'].values)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in df_train['preprocess_text'].values])

x_train = pad_sequences(sequences_train, maxlen=training_length)
x_valid = pad_sequences(sequences_val, maxlen=training_length)

In [36]:
word_count, training_length

(127209, 30)

In [37]:
#@title Определим глобальные параметры для архитектур
y_train = df_train['class'].values
y_val = df_val['class'].values

batch_size = 512
epochs = 10
validation_split = 0.1
verbose = 1
early_stopping = EarlyStopping(monitor='val_loss')

# Для вывода в дальнейшем
results = {
    "NN": [],
    "loss": [],
    "accuracy": []
}

## 1. Построим свёрточные архитектуры

In [41]:
model = create_model('CNN')
score = fit_evalute_model(model, x_train, y_train, x_valid, y_val)

results['NN'].append("CNN")
results['loss'].append(score[0])
results['accuracy'].append(score[1])

Epoch 1/10
319/319 [==============================] - 30s 91ms/step - loss: 0.5909 - accuracy: 0.6767 - val_loss: 0.5486 - val_accuracy: 0.7159
Epoch 2/10
319/319 [==============================] - 34s 106ms/step - loss: 0.4539 - accuracy: 0.7895 - val_loss: 0.5662 - val_accuracy: 0.7115
Epoch 3/10
319/319 [==============================] - 40s 124ms/step - loss: 0.3194 - accuracy: 0.8638 - val_loss: 0.6321 - val_accuracy: 0.7082
Epoch 4/10
319/319 [==============================] - 36s 113ms/step - loss: 0.2199 - accuracy: 0.9092 - val_loss: 0.7274 - val_accuracy: 0.6982
Epoch 5/10
319/319 [==============================] - 38s 121ms/step - loss: 0.1593 - accuracy: 0.9356 - val_loss: 0.8407 - val_accuracy: 0.6955
Epoch 6/10
319/319 [==============================] - 34s 107ms/step - loss: 0.1211 - accuracy: 0.9522 - val_loss: 0.9574 - val_accuracy: 0.6885
Epoch 7/10
319/319 [==============================] - 42s 131ms/step - loss: 0.0933 - accuracy: 0.9638 - val_loss: 1.0849 - val_acc

## 2. Построим различные архитектуры с RNN

In [50]:
for name_arch in [SimpleRNN, LSTM, GRU]:
  print(f"start work with model: {name_arch.__name__}\n")
  model = create_model(name_arch)
  score = fit_evalute_model(model, x_train, y_train, x_valid, y_val)
  print(f"\nend work with model: {name_arch.__name__}\n")

  results["NN"].append(name_arch.__name__)
  results["loss"].append(score[0])
  results["accuracy"].append(score[1])

start work with model: SimpleRNN

Epoch 1/10
319/319 [==============================] - 44s 133ms/step - loss: 0.6013 - accuracy: 0.6628 - val_loss: 0.5492 - val_accuracy: 0.7189
Epoch 2/10
319/319 [==============================] - 49s 155ms/step - loss: 0.4384 - accuracy: 0.8011 - val_loss: 0.5770 - val_accuracy: 0.7095
Epoch 3/10
319/319 [==============================] - 38s 120ms/step - loss: 0.2732 - accuracy: 0.8870 - val_loss: 0.6915 - val_accuracy: 0.6991
Epoch 4/10
319/319 [==============================] - 43s 135ms/step - loss: 0.1861 - accuracy: 0.9236 - val_loss: 0.8305 - val_accuracy: 0.6851
Epoch 5/10
319/319 [==============================] - 36s 113ms/step - loss: 0.1482 - accuracy: 0.9387 - val_loss: 0.9446 - val_accuracy: 0.6862
Epoch 6/10
319/319 [==============================] - 34s 108ms/step - loss: 0.1272 - accuracy: 0.9459 - val_loss: 1.0749 - val_accuracy: 0.6872
Epoch 7/10
319/319 [==============================] - 34s 107ms/step - loss: 0.1125 - accuracy: 

## 3. Построим совместные архитектуры CNN -> RNN и RNN -> CNN

In [54]:
for name_arch in ['RNN+CNN', 'CNN+RNN']:
  for rnn_method in [SimpleRNN, LSTM, GRU]:
    print(f"start work with model: {name_arch} with rnn_method: {rnn_method.__name__}\n")
    model = create_model(name_arch, rnn_method=rnn_method)
    score = fit_evalute_model(model, x_train, y_train, x_valid, y_val, callbacks=[early_stopping])
    print(f"\nend work with model: {name_arch} with rnn_method: {rnn_method.__name__}\n")

    results["NN"].append(f"{name_arch} - RNN method: {rnn_method.__name__}")
    results["loss"].append(score[0])
    results["accuracy"].append(score[1])

start work with model: RNN+CNN with rnn_method: SimpleRNN

Epoch 1/10
319/319 [==============================] - 47s 137ms/step - loss: 0.6016 - accuracy: 0.6606 - val_loss: 0.5487 - val_accuracy: 0.7158
Epoch 2/10
319/319 [==============================] - 44s 137ms/step - loss: 0.4352 - accuracy: 0.7994 - val_loss: 0.5691 - val_accuracy: 0.7148
Epoch 3/10
319/319 [==============================] - 44s 138ms/step - loss: 0.2743 - accuracy: 0.8844 - val_loss: 0.6868 - val_accuracy: 0.7037
Epoch 4/10
319/319 [==============================] - 43s 134ms/step - loss: 0.1894 - accuracy: 0.9215 - val_loss: 0.7886 - val_accuracy: 0.6915
Epoch 5/10
319/319 [==============================] - 42s 133ms/step - loss: 0.1503 - accuracy: 0.9376 - val_loss: 0.8992 - val_accuracy: 0.6883
Epoch 6/10
319/319 [==============================] - 42s 133ms/step - loss: 0.1298 - accuracy: 0.9462 - val_loss: 1.0032 - val_accuracy: 0.6826
Epoch 7/10
319/319 [==============================] - 44s 137ms/step - 

## Результат

In [55]:
pd.DataFrame(results)

,NN,loss,accuracy
0,CNN,1.513075,0.676806
1,SimpleRNN,1.718922,0.672221
2,LSTM,1.424162,0.679584
3,GRU,1.471497,0.682934
4,RNN+CNN - RNN method: SimpleRNN,1.239855,0.669665
5,RNN+CNN - RNN method: LSTM,1.014610,0.681656
6,RNN+CNN - RNN method: GRU,1.311073,0.683463
7,CNN+RNN - RNN method: SimpleRNN,1.622373,0.675352
8,CNN+RNN - RNN method: LSTM,1.458165,0.673500
9,CNN+RNN - RNN method: GRU,1.623200,0.678173
